# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [4]:
#| hide
from nbdev.showdoc import *

In [11]:
#| export 
import httpx
import zipfile
from fastcore.all import *
from fastcore.utils import *
import pandas as pd

In [12]:
#| hide 
df = pd.read_csv("company2.csv", index_col=0)
df.head()

,TATAMOTORS,ASHOKLEY,OLECTRA,FORCEMOT,SMLISUZU
Financial Year 2024,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....
Financial Year 2023,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....
Financial Year 2022,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
Financial Year 2021,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
Financial Year 2020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...


In [13]:
#| export
def make_dirs(base:Path, df:pd.core.frame.DataFrame):
    """
    Takes in base directory make all the directory for given dataframe wrt index and and columns in hierarchy
    Example:
    If df has columns ['A', 'B'] and index ['X', 'Y'], the following structure is created:
    
    base/
    ├── A/
    │   ├── X/
    │   └── Y/
    └── B/
        ├── X/
        └── Y/
    """
    base.mkdir(exist_ok=True)
    for c in df.columns:
        dir = base/c
        dir.mkdir(exist_ok=True)
        for i in df.index:
            if "Right " not in i:
                (dir/i).mkdir(exist_ok=True)

## download url and rename

In [14]:
dir = Path("result")
make_dirs(dir, df)

In [15]:
#| export 
import httpx
async def download_url(url:str, fn:Path):
    """
    download given url and write it a given fn.
    """
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/pdf,*/*'
    }
    timeout = httpx.Timeout(connect=20.0, read=120.0, write=30.0, pool=60.0)

    async with httpx.AsyncClient(timeout=30.0, follow_redirects=True) as client:
        try:
            res = await client.get(url, headers=headers)
            if res.status_code == 200:
                with open(fn, 'wb') as f:
                    f.write(res.content)
                return True
            else:
                print(f"Error downloading: Status code {res.status_code}")
                return False
        except httpx.RequestError as e:
            print(f"Request failed: {e}")
            return False

In [ ]:
url = "https://www.bseindia.com/xml-data/corpfiling/AttachHis/0913b647-b205-4a3c-ad9b-8c493f97f972.pdf"
fn = dir/"temp.pdf"
await download_url(url, fn)
assert fn.is_file()

True

In [ ]:
url = "https://archives.nseindia.com/annual_reports/AR_ASHOKLEY_2009_2010_18082010121500.zip"
fn = dir/"temp.zip"
await download_url(url, fn)
assert fn.is_file() 

In [16]:
#| hide
fn = dir/"1.pdf"
url = "https://static-assets.tatamotors.com/Production/www-tatamotors-com-NEW/wp-content/uploads/2024/05/tata-motor-IAR-2023-24.pdf"
await download_url(url, fn)
assert fn.is_file()

In [18]:
#| export
def rn_zip_extract(zipf, fn):
    """
    Donwload and extract the given files with filename fn 
    """
    home = Path(zipf).parent
    with zipfile.ZipFile(zipf, 'r') as zip_ref:
        f = zip_ref.namelist()[0]
        zip_ref.extractall(path=home)
        fi = home/f
        fi.rename(fn)

In [ ]:
p = dir/"i.pdf"
rn_zip_extract(fn, p)
assert p.is_file()


In [ ]:
#| hide
!rm -rf result/*.pdf result/*.zip

In [19]:
#| export 
async def download_rename(url, fn, dir):
    """
    download given url and rename the file only handles zip and pdf
    """
    fn, dir = Path(fn), Path(dir)
    dir.mkdir(exist_ok=True)

    if ".zip" in url:
        zip = dir/"temp.zip"
        await download_url(url, zip)
        rn_zip_extract(zip, fn)
        Path(dir/"temp.zip").unlink()
    elif ".pdf" in url:
        await download_url(url, fn)

In [ ]:
await download_rename(url, fn, dir)
assert fn.is_file()

## Pdf functions

In [20]:
#| export
import PyPDF2

def pdfPC(pth):
    """
    given pdf file name returns the no of pages 
    """
    with open(pth, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        return len(reader.pages)

In [21]:
PC = pdfPC(fn)
assert PC != 0

In [22]:
#| export 
from pdf2image import convert_from_path
from PIL.PpmImagePlugin import PpmImageFile
from typing import Tuple, Callable, List, Optional
from PIL import Image

In [23]:
#| export 
def pdf2imgOffset(
    fn: str, 
    limit: Tuple[int, int], 
    transform: Optional[Callable[[Image.Image], Image.Image]] = None,
    dir:Optional[Path] = None) -> List[Image.Image]:
    """
    given a pdf file it `fn` and limit -> [first_page, last_page] it return pdf in image
    transform: performs the operation on a given image if given
    dir: save the images to a file
    """
    fp, lp = limit
    ims =  convert_from_path(str(fn), dpi=250, first_page=fp, last_page=lp)
    
    if transform is not None:
        ims = [transform(i) for i in ims]

    if dir:
        dir.mkdir(exist_ok=True)
        [im.save(dir/"{0}.png".format(fp+i)) for i, im in enumerate(ims) ]  
    else:  
        return ims

In [24]:
#| export 
def devide_img(im:Image, n:float=0.5):
    """
    Devide the image by factor `n`
    """
    w, h = im.size
    im.thumbnail((int(w*n), int(h*n) ), Image.LANCZOS)
    return im

In [ ]:
im1 = pdf2imgOffset(fn, (0, 10)) # reads first 10 pages
assert type(im1) == list
assert len(im1) == 10

In [ ]:
im2 = pdf2imgOffset(fn, (0, 10), devide_img) # read first 10 pages then dived the image by factor of 0.5
assert all([im2[0].size[0] * .5 <= im1[0].size[0], im2[0].size[1] * 0.5 <= im1[0].size[1]])

In [ ]:
im2 = pdf2imgOffset(fn, (0, 10), devide_img, dir) # read 10 pages, device the image and save it to dir
assert all([(dir/f"{i}.png").is_file() for i in range(10)])

In [ ]:
#| hide
[(dir/f"{i}.png").unlink() for i in range(10)]

[None, None, None, None, None, None, None, None, None, None]

In [25]:
#| export 

from math import ceil

def pdf2img(_process_chunk:Callable, pdf_path:Path, offset:int, n_workers:int=4):
    
    PC = pdfPC(pdf_path)
    li = [(0, offset) if i == 0 else (i * offset + 1, i * offset + offset) for i in range(ceil(PC/offset))]
    
    return parallel(_process_chunk, li, progress=True, n_workers=n_workers), li

In [26]:
#| export 

def _process_chunk(offset):
    return pdf2imgOffset(fn=fn, limit=offset, transform=devide_img, dir=dir)

In [ ]:
ims, li = pdf2img(_process_chunk, fn, 10)
assert all([all([(dir/f"{i}.png").is_file() for i in range(a[0], a[1])]) for a in li])

[(0, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60), (61, 70), (71, 80), (81, 90), (91, 100), (101, 110), (111, 120), (121, 130), (131, 140), (141, 150), (151, 160), (161, 170), (171, 180), (181, 190), (191, 200), (201, 210), (211, 220), (221, 230), (231, 240), (241, 250), (251, 260), (261, 270), (271, 280), (281, 290), (291, 300), (301, 310), (311, 320), (321, 330), (331, 340), (341, 350), (351, 360), (361, 370), (371, 380), (381, 390), (391, 400), (401, 410), (411, 420), (421, 430), (431, 440), (441, 450), (451, 460), (461, 470), (471, 480), (481, 490), (491, 500), (501, 510), (511, 520), (521, 530)]


In [27]:
#| export 

def resize_image_if_needed(im:Image.Image, limit:int=1280*28*28 ):
    """
    check if the image of size `1280*28*28` which is the upper limit of image can be processed by 
    """
    w, h = im.size

    sf = math.sqrt( limit/(w*h))
    if sf < 1.:
        nw = int(w * sf)
        nh = int(h * sf)
        im = im.resize((nw,nh), Image.LANCZOS)

    return im

In [23]:
im2 = pdf2imgOffset(fn, (0, 10), resize_image_if_needed, dir) # read 10 pages, device the image and save it to dir
assert all([(dir/f"{i}.png").is_file() for i in range(10)])

In [ ]:
#| hide
!rm result/*.png

## Save file to md file

In [ ]:
#| export 
def save_md(fn: str, txt: str):
    """
    Writes the given text to a markdown file as binary (UTF-8 encoded).
    Parameters:
    fn (str): Filename or path to save the markdown file.
    txt (str): Content to write to the file.
    """
    try:
        # Open the file in binary write mode and encode the text to bytes
        with open(fn, 'wb') as f:
            f.write(txt.encode('utf-8'))
    except Exception as e:
        print(f"Error writing to file {fn}: {e}")
    

In [ ]:
fn = dir/"temp.md"
save_md(fn, "# h1\n- 1\n- 2")
assert fn.is_file()

In [4]:
#| export 
from IPython.display import display, Markdown
def show_markdown(md_text: str):
    """
    Displays Markdown content in a Jupyter Notebook.
    Args:
        md_text (str): The Markdown-formatted string to display.
    """
    display(Markdown(md_text))

In [2]:
#| export
import re

def str2md(text: str) -> str:
    """
    Extracts the Markdown content from a string that contains a markdown code block.

    The function looks for a code block that starts with:
        ```markdown
    and ends with:
        ```
    
    Args:
        text (str): The input string containing a markdown code block.

    Returns:
        str: The extracted Markdown content if found; otherwise, an empty string.
    """
    # Regex pattern to capture content between ```markdown and ```
    pattern = re.compile(r"```markdown\s*(.*?)\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1)
    return ""

In [5]:
# Example usage:
txt = """
Some introductory text.

```markdown
# This is a Markdown Title

- List item 1
- List item 2

**Bold Text**
```"""
show_markdown(str2md(txt))

# This is a Markdown Title

- List item 1
- List item 2

**Bold Text**

In [ ]:
#| hide
!rm -rf  result/

## Export data

In [ ]:
#| hide
async setup_dir(df, dir):
    make_dirs(dir, df)
    for c in df.columns:
        b_dir = dir/c
        print(c)
        for i in df.index:
            print(df.loc[i][c])
            if "Right " not in i:
                f_dir = b_dir/i
                fn = f_dir/"index.pdf"
                await download_rename(df.loc[i][c], fn, f_dir)
#await setup(df, Path("/mnt/d/finfox/"))

In [29]:
#| hide

from functools import partial
#func = partial(devide_img, n=0.35)

dir = Path("/mnt/d/finfox/")
def _process_chunk(offset):
        return pdf2imgOffset(fn=pdf, limit=offset, transform=resize_image_if_needed, dir=f_dir)


for c in ['SMLISUZU'] : #df.columns:
    b_dir = dir/c
    print(b_dir)
    for i in df.index:
        print(i, end=" ")
        if "Right " not in i:
                f_dir = b_dir/i/"index_img"
                pdf = b_dir/i/"index.pdf"
                pdf2img(_process_chunk, pdf, 10)

/mnt/d/finfox/SMLISUZU
Financial Year 2024 

Financial Year 2023 

Financial Year 2022 

Financial Year 2021 

Financial Year 2020 

...

Financial Year 2014 

Financial Year 2013 

Financial Year 2012 

Financial Year 2011 

Financial Year 2010 

In [ ]:
#| hide
def im_files(dir):
    ims = L(str(i) for i in L(dir.glob('**/*.png')))
    pd = pd.DataFrame({"fn" : ims})
    pd.to_csv("imgs.csv", index=False)

#im_file(dir)


In [30]:
#| hide
import nbdev; nbdev.nbdev_export()